#ServerFile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install diffusers transformers accelerate safetensors torch torchvision
!mkdir images

In [ ]:
!pip install fastapi uvicorn

In [3]:
from pydantic import BaseModel
from diffusers import StableDiffusionXLPipeline
import torch
import os, shutil
from PIL import Image
from IPython.display import display
from fastapi import FastAPI, File, UploadFile, Form
from random import randint
import datetime


In [4]:
%%writefile main.py
from pydantic import BaseModel
from diffusers import StableDiffusionXLImg2ImgPipeline
import torch
import os, shutil
from PIL import Image
from fastapi import FastAPI, File, UploadFile, Form
from random import randint
import datetime

from fastapi.staticfiles import StaticFiles

app = FastAPI()
app.mount("/images", StaticFiles(directory="images"), name="images")


# Load Model
my_model_path_in_drive = ""

pipe_img = StableDiffusionXLImg2ImgPipeline.from_single_file(
    my_model_path_in_drive,
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

@app.post("/generar")
async def generar_img(
    prompt: str = Form(...),
    negative_prompt: str = Form(""),
    strength: float = Form(0.75),
    image: UploadFile = File(...)
):
    #  UploadFile
    init_image = Image.open(image.file).convert("RGB").resize((1024, 1024))

    # gen img
    result = pipe_img(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=strength,
        guidance_scale=5,
        num_inference_steps=40
    ).images[0]

    date = datetime.datetime.now()
    date_str = date.strftime("%Y-%m-%d_%H-%M-%S")
    image_name = f"{randint(0, 100)}_{date_str}.png"
    os.makedirs("images", exist_ok=True)
    image_path = os.path.join("images", image_name)
    result.save(image_path)

    return {"mensaje": "Imagen generada", "file_url": image_name}


Writing main.py


In [ ]:
!uvicorn main:app --reload

**configure ngrok tunnel**

open the google colab terminal and run:

curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok

ngrok config add-authtoken *Your Token*

ngrok http http://localhost:8000

*

In [ ]:
"""
pipe_img = StableDiffusionXLImg2ImgPipeline.from_single_file(
    '',
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")
"""


In [ ]:
"""
pipe = StableDiffusionXLPipeline.from_single_file(
    ''
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

"""
